In [22]:
# Import libraries 
from dash import Dash, html, dcc, callback, Output, Input
import dash_bootstrap_components as dbc
import plotly.express as px
import pandas as pd

## DATA

In [11]:
df = pd.read_csv('sample_superstore.csv', encoding='windows-1252', delimiter=';')
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,08/11/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,08/11/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,Hon Deluxe Fabric Upholstered Stacking Chairs ...,731.9400,3,0.00,219.5820
2,3,CA-2016-138688,12/06/2016,16/06/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,11/10/2015,18/10/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,11/10/2015,18/10/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         9994 non-null   int64  
 1   Order ID       9994 non-null   object 
 2   Order Date     9994 non-null   object 
 3   Ship Date      9994 non-null   object 
 4   Ship Mode      9994 non-null   object 
 5   Customer ID    9994 non-null   object 
 6   Customer Name  9994 non-null   object 
 7   Segment        9994 non-null   object 
 8   Country        9994 non-null   object 
 9   City           9994 non-null   object 
 10  State          9994 non-null   object 
 11  Postal Code    9994 non-null   int64  
 12  Region         9994 non-null   object 
 13  Product ID     9994 non-null   object 
 14  Category       9994 non-null   object 
 15  Sub-Category   9994 non-null   object 
 16  Product Name   9994 non-null   object 
 17  Sales          9994 non-null   float64
 18  Quantity

In [14]:
df['Order Date'] = pd.to_datetime(df['Order Date'], dayfirst=True)

In [ ]:
df['Order Month'] = df['Order Date'].dt.month
df['Order Year'] = df['Order Date'].dt.year


In [21]:
df['Sub-Category'].unique()

array(['Bookcases', 'Chairs', 'Labels', 'Tables', 'Storage',
       'Furnishings', 'Art', 'Phones', 'Binders', 'Appliances', 'Paper',
       'Accessories', 'Envelopes', 'Fasteners', 'Supplies', 'Machines',
       'Copiers'], dtype=object)

## Dash App

In [40]:
# Initialize dash app with stylesheet
app = Dash(external_stylesheets=[dbc.themes.FLATLY])

app.layout = html.Div(children = [

    # Dashboard Title
    html.H1('Superstore Sales Dashboard', style={'text-align': 'center'}),
    
    # Dropdown for Region selection (optional)
    html.Label('Select Region (optional)'),
    dcc.Dropdown(
        id='region-dropdown',
        options=[{'label': region, 'value': region} for region in df['Region'].unique()],
        value=None,  
        clearable=True
    ),
    
    # Dropdown for Category selection (optional)
    html.Label('Select Category (optional)'),
    dcc.Dropdown(
        id='category-dropdown',
        options=[{'label': category, 'value': category} for category in df['Category'].unique()],
        value=None,
        clearable=True
    ),
    
    # Display total sales and profit
    html.Div(id='total-sales', style={'fontSize': 24, 'fontWeight': 'bold'}),
    html.Div(id='total-profit', style={'fontSize': 24, 'fontWeight': 'bold'}),

    # Line chart for monthly sales
    dcc.Graph(id='monthly-sales-line-chart'),
    
    # Bar chart for sales per category or sub-category
    dcc.Graph(id='category-bar-chart')
],
className = 'bg-secondary dbc',
style = {'padding' : 10}
)

# Callback for total sales and profit
@app.callback(
    [Output('total-sales', 'children'),
     Output('total-profit', 'children')],
    [Input('region-dropdown', 'value'),
     Input('category-dropdown', 'value')]
)
def update_totals(region, category):
    filtered_df = df
    if region:
        filtered_df = filtered_df[filtered_df['Region'] == region]
    if category:
        filtered_df = filtered_df[filtered_df['Category'] == category]
    
    total_sales = filtered_df['Sales'].sum()
    total_profit = filtered_df['Profit'].sum()
    return (f"Total Sales: ${total_sales:,.2f}", f"Total Profit: ${total_profit:,.2f}")

# Callback for monthly sales line chart
@app.callback(
    Output('monthly-sales-line-chart', 'figure'),
    [Input('region-dropdown', 'value'),
     Input('category-dropdown', 'value')]
)
def update_monthly_sales_chart(region, category):
    filtered_df = df
    if region:
        filtered_df = filtered_df[filtered_df['Region'] == region]
    if category:
        filtered_df = filtered_df[filtered_df['Category'] == category]
    
    monthly_sales = filtered_df.groupby(['Order Year', 'Order Month'])['Sales'].sum().reset_index()
    fig = px.line(monthly_sales, x='Order Month', y='Sales', color='Order Year', title='Monthly Sales')
    fig.update_layout(xaxis_title='Month', yaxis_title='Sales')
    return fig

# Callback for category/sub-category bar chart
@app.callback(
    Output('category-bar-chart', 'figure'),
    [Input('region-dropdown', 'value'),
     Input('category-dropdown', 'value')]
)
def update_category_chart(region, category):
    filtered_df = df
    if region:
        filtered_df = filtered_df[filtered_df['Region'] == region]
    
    if category:
        category_sales = filtered_df[filtered_df['Category'] == category].groupby('Sub-Category')['Sales'].sum().reset_index()
        fig = px.bar(category_sales, x='Sub-Category', y='Sales', title=f'Sales per Sub-Category in {category}', color_discrete_sequence=['#18bc9c'])
    else:
        category_sales = filtered_df.groupby('Category')['Sales'].sum().reset_index()
        fig = px.bar(category_sales, x='Category', y='Sales', title='Sales per Category', color_discrete_sequence=['#18bc9c'])
    
    fig.update_layout(xaxis_title='Category' if not category else 'Sub-Category', yaxis_title='Sales')
    return fig

if __name__ == '__main__':
    app.run(debug=True, jupyter_mode = 'external')

Dash app running on http://127.0.0.1:8050/
